In [32]:
import string
import numpy as np

In [36]:
class HangMan:
    def __init__(self):
        self.guessed_letters = []
        self.len_incorrect_word = 0
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.current_dictionary = self.full_dictionary

    def guess(self, word):

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word.replace("_", ".")
        clean_word = clean_word.replace(" ", "")

        # find incorrect letters by comparing guessed letters with the word
        incorrect_letters = set(self.guessed_letters) - set(word)

        # make a new dictionary that only contains words that does not contain any of the incorrect letters
        new_dictionary = [dict_word for dict_word in self.current_dictionary 
                          if not any((letter in incorrect_letters) for letter in dict_word)]
        
        # update the current dictionary of self with new dictionary
        self.current_dictionary = new_dictionary

        # now, if the model is doing correct predictions then we don't need to update the n-gram models
        # so check if the number of incorrect letters in last guess is equal to the number of incorrect
        # letters in the current guess
        if (len(incorrect_letters) != self.len_incorrect_word) or (len(self.guessed_letters)==0):
            self.len_incorrect_word = len(incorrect_letters)
            self.build_ngram_models()
        
        # before calculating probabilites of all characters initialize the probabilities to 0
        # here, indices 0-25 correspond to a-z characters and the value represents the probability of that character
        self.probabilities = [0]*26  

        # calculate the probabilities of all characters
        self.five_gram_probability(clean_word, 0.87)
        self.four_gram_probability(clean_word, 0.71)
        self.three_gram_probability(clean_word, 0.58)
        self.two_gram_probability(clean_word, 0.4)
        self.one_gram_probability(clean_word, 0.19)

        # if the sum of probabilities is 0 then we have no information about the word
        # so we will return a random character but we will make sure that the character is not already guessed
        # this case might occur because we are removing words from the dictionary that contain incorrect letters
        # and we might end up with a dictionary that does not contain any word
        if sum(self.probabilities) == 0:
            for character in "esiarntolcdupmghbyfvkwzxqj":  # sequence of characters based on their frequency in English language
                if character not in self.guessed_letters:
                    return character
        
        # sort the probabilities in descending order
        sorted_probabilities = list(np.argsort(self.probabilities))  
        sorted_probabilities.reverse()

        # return the character with the highest probability that is not already guessed
        for character_index in sorted_probabilities:
            if string.ascii_lowercase[character_index] not in self.guessed_letters:
                return string.ascii_lowercase[character_index]
        
        assert False, "Should not reach here"

    def build_ngram_models(self):

        # find maximum length of a word in the dictionary, we need this because we
        # will be using 1-gram and 2-gram models based on the length of the word
        max_word_length = len(max(self.current_dictionary, key=len))

        # initialize the n-gram models

        # 1-gram model for each word length
        self.one_gram = {n: {character: 0 for character in string.ascii_lowercase} for n in range(1, max_word_length+1)}

        for dict_word in self.current_dictionary:
            # here we are just checking that how frequently a character appears in the words of the dictionary
            n = len(dict_word)
            for character in set(dict_word):
                self.one_gram[n][character] += 1

        # 2-gram model for each word length
        self.two_gram = {n: {character: {character: 0 for character in string.ascii_lowercase}
                             for character in string.ascii_lowercase} 
                         for n in range(1, max_word_length + 1)}

        for dict_word in self.current_dictionary:
            n = len(dict_word)
            for i in range(n-1):
                self.two_gram[n][dict_word[i]][dict_word[i+1]] += 1
        
        # 3-gram model for each word length
        self.three_gram = {n: {character: {character: {character: 0 for character in string.ascii_lowercase}
                                           for character in string.ascii_lowercase}
                               for character in string.ascii_lowercase}
                           for n in range(1, max_word_length + 1)}
        
        for dict_word in self.current_dictionary:
            n = len(dict_word)
            for i in range(n-2):
                self.three_gram[n][dict_word[i]][dict_word[i+1]][dict_word[i+2]] += 1
        
        # 4-gram model without considering the word length
        self.four_gram = {character: {character: {character: {character:0 for character in string.ascii_lowercase} 
                                                  for character in string.ascii_lowercase} 
                                      for character in string.ascii_lowercase}
                          for character in string.ascii_lowercase}
        
        for dict_word in self.current_dictionary:
            for i in range(len(dict_word)-3):
                self.four_gram[dict_word[i]][dict_word[i+1]][dict_word[i+2]][dict_word[i+3]] += 1
        
        # 5-gram model without considering the word length
        self.five_gram = {character: {character: {character: {character: {character:0 for character in string.ascii_lowercase} 
                                                              for character in string.ascii_lowercase} 
                                                  for character in string.ascii_lowercase}
                                      for character in string.ascii_lowercase}
                          for character in string.ascii_lowercase}
        
        for dict_word in self.current_dictionary:
            for i in range(len(dict_word)-4):
                self.five_gram[dict_word[i]][dict_word[i+1]][dict_word[i+2]][dict_word[i+3]][dict_word[i+4]] += 1
    
    def five_gram_probability(self, word, weight):
        # inititalize the probabilities to 0
        # note that we are not updating the probabilities that the object already contains
        # but we are making a temporary list of probabilities that we will contain probabilities
        # given by the 5-gram model
        temp_probabilities = [0]*26

        # iterate through the word and calculate the probabilities based on the 5-gram model
        for i in range(len(word)-4):

            # check if the word is in the form of . * * * *
            # where * is any character that is known
            # and . is the character that we want to predict
            if word[i]=="." and word[i+1]!="." and word[i+2]!="." and word[i+3]!="." and word[i+4]!=".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.five_gram[character][word[i+1]][word[i+2]][word[i+3]][word[i+4]]

            # check if the word is in the form of * . * * *
            if word[i]!="." and word[i+1]=="." and word[i+2]!="." and word[i+3]!="." and word[i+4]!=".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.five_gram[word[i]][character][word[i+2]][word[i+3]][word[i+4]]

            # check if the word is in the form of * * . * *
            if word[i]!="." and word[i+1]!="." and word[i+2]=="." and word[i+3]!="." and word[i+4]!=".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.five_gram[word[i]][word[i+1]][character][word[i+3]][word[i+4]]

            # check if the word is in the form of * * * . *
            if word[i]!="." and word[i+1]!="." and word[i+2]!="." and word[i+3]=="." and word[i+4]!=".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.five_gram[word[i]][word[i+1]][word[i+2]][character][word[i+4]]
            
            # check if the word is in the form of * * * * .
            if word[i]!="." and word[i+1]!="." and word[i+2]!="." and word[i+3]!="." and word[i+4]==".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.five_gram[word[i]][word[i+1]][word[i+2]][word[i+3]][character]
            
        # update the probabilites of all characters in the object's probabilities list
        sum_temp_probabilities = sum(temp_probabilities)
        if sum_temp_probabilities != 0:
            for j in range(len(self.probabilities)):
                self.probabilities[j] += ((temp_probabilities[j]*weight)/sum_temp_probabilities)
        
    def four_gram_probability(self, word, weight):
        # again make a temporary list of probabilities that we will update based on the 4-gram model
        temp_probabilities = [0]*26
        
        # iterate through the word and calculate the probabilities based on the 4-gram model
        for i in range(len(word)-3):
            # check if the word is in the form of . * * *
            # where * is any character and . is the character that we want to predict
            if word[i]=="." and word[i+1]!="." and word[i+2]!="." and word[i+3]!=".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.four_gram[character][word[i+1]][word[i+2]][word[i+3]]
            
            # check if the word is in the form of * . * *
            if word[i]!="." and word[i+1]=="." and word[i+2]!="." and word[i+3]!=".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.four_gram[word[i]][character][word[i+2]][word[i+3]]
            
            # check if the word is in the form of * * . *
            if word[i]!="." and word[i+1]!="." and word[i+2]=="." and word[i+3]!=".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.four_gram[word[i]][word[i+1]][character][word[i+3]]

            # check if the word is in the form of * * * .
            if word[i]!="." and word[i+1]!="." and word[i+2]!="." and word[i+3]==".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.four_gram[word[i]][word[i+1]][word[i+2]][character]

        # update the probabilites of all characters in the object's probabilities list
        sum_temp_probabilities = sum(temp_probabilities)
        if sum_temp_probabilities != 0:
            for j in range(len(self.probabilities)):
                self.probabilities[j] += ((temp_probabilities[j]*weight)/sum_temp_probabilities)
            
    def three_gram_probability(self, word, weight):
        # make a temporary list of probabilities that we will update based on the 3-gram model
        temp_probabilities = [0]*26
        
        # As we just want to check the probabilities based on the length of the word we have to check
        # the length and then update the probabilities based on the 3-gram model
        n = len(word)

        # check whether there exists any key in the 3-gram model for the length of the word
        # if not just check the nearest key in the 3-gram model
        if n not in self.three_gram:
            check = False
            for i in range(1,3):
                if n+i in self.three_gram:
                    n = n+i
                    check = True
                    break
                if n-i in self.three_gram:
                    n = n-i
                    check = True
                    break
            if not check:
                return

        # iterate through the word and calculate the probabilities based on the 3-gram model
        for i in range(len(word)-2):
            # check if the word is in the form of . * *
            # where * is any character and . is the character that we want to predict
            if word[i]=="." and word[i+1]!="." and word[i+2]!=".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.three_gram[n][character][word[i+1]][word[i+2]]
            
            # check if the word is in the form of * . *
            if word[i]!="." and word[i+1]=="." and word[i+2]!=".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.three_gram[n][word[i]][character][word[i+2]]
            
            # check if the word is in the form of * * .
            if word[i]!="." and word[i+1]!="." and word[i+2]==".": 
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.three_gram[n][word[i]][word[i+1]][character]
            
        # update the probabilites of all characters in the object's probabilities list
        sum_temp_probabilities = sum(temp_probabilities)
        if sum_temp_probabilities != 0:
            for j in range(len(self.probabilities)):
                self.probabilities[j] += ((temp_probabilities[j]*weight)/sum_temp_probabilities)
        
    def two_gram_probability(self, word, weight):
        # make a temporary list of probabilities that we will update based on the 2-gram model
        temp_probabilities = [0]*26
        
        # As we just want to check the probabilities based on the length of the word we have to check
        # the length and then update the probabilities based on the 2-gram model
        n = len(word)

        # check whether there exists any key in the 2-gram model for the length of the word
        if n not in self.two_gram:
            return

        # iterate through the word and calculate the probabilities based on the 2-gram model
        for i in range(len(word)-1):
            # check if the word is in the form of . *
            # where * is any character and . is the character that we want to predict
            if word[i]=="." and word[i+1]!=".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.two_gram[n][character][word[i+1]]
            
            # check if the word is in the form of * .
            if word[i]!="." and word[i+1]==".":
                for j, character in enumerate(string.ascii_lowercase):
                    if character not in self.guessed_letters:
                        temp_probabilities[j] += self.two_gram[n][word[i]][character]
        
        # update the probabilites of all characters in the object's probabilities list
        sum_temp_probabilities = sum(temp_probabilities)
        if sum_temp_probabilities != 0:
            for j in range(len(self.probabilities)):
                self.probabilities[j] += ((temp_probabilities[j]*weight)/sum_temp_probabilities)

    def one_gram_probability(self, word, weight):
        # make a temporary list of probabilities that we will update based on the 1-gram model
        temp_probabilities = [0]*26
        
        # As we just want to check the probabilities based on the length of the word we have to check
        # the length and then update the probabilities based on the 1-gram model
        n = len(word)

        # check whether there exists any key in the 1-gram model for the length of the word
        if n not in self.one_gram:
            return

        # update the probabilities based on the 1-gram model
        for j, character in enumerate(string.ascii_lowercase):
            if character not in self.guessed_letters:
                temp_probabilities[j] += self.one_gram[n][character]

        # update the probabilites of all characters in the object's probabilities list
        sum_temp_probabilities = sum(temp_probabilities)
        if sum_temp_probabilities != 0:
            for j in range(len(self.probabilities)):
                self.probabilities[j] += ((temp_probabilities[j]*weight)/sum_temp_probabilities)


    def is_complete(self, word):
        for character in word:
            if character == "_":
                return 0
        return 1

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
    
    def play_game(self, word, computer=True):
        self.guessed_letters = []
        self.len_incorrect_word = 0
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.current_dictionary = self.full_dictionary
        self.build_ngram_models()
        word_to_be_printed = "_ "*len(word)
        tries_left = 6
        if not computer:
            import random
            word = random.choice(self.full_dictionary)

        while tries_left>0:
            print("Word: ", word_to_be_printed)
            print("Guesses left: ", tries_left)
            print("Number of tries left: ", tries_left)
            if computer:
                guess = self.guess(word_to_be_printed)
                print("Computer's guess: ", guess)
            else:
                guess = input("Enter your guess: ")
            self.guessed_letters.append(guess)
            if guess in word:
                temp = ""
                for i in range(len(word)):
                    if guess == word[i]:
                        temp += guess + " "
                    else:
                        temp += word_to_be_printed[2*i] + " "
                word_to_be_printed = temp
            else:
                tries_left -= 1
            print(u'\u2500' * 100)
            if self.is_complete(word_to_be_printed):
                print("Congratulations! You have guessed the word: ", word)
                break
        if tries_left == 0:
            print("You have lost! The word was: \"", word, "\" and you guessed: ", word_to_be_printed)
            print("Game Over!")

In [37]:
hangman = HangMan()
hangman.play_game("practice", computer=True) # if you want to try then just do computer = false 
                                                    # and give practice as a word which will give you 
                                                    # a practice word and example and when you do computer = False
                                                    # then a random word will be selected from the dictionary

Word:  _ _ _ _ _ _ _ _ 
Guesses left:  6
Number of tries left:  6
Computer's guess:  e
────────────────────────────────────────────────────────────────────────────────────────────────────
Word:  _ _ _ _ _ _ _ e 
Guesses left:  6
Number of tries left:  6
Computer's guess:  r
────────────────────────────────────────────────────────────────────────────────────────────────────
Word:  _ r _ _ _ _ _ e 
Guesses left:  6
Number of tries left:  6
Computer's guess:  a
────────────────────────────────────────────────────────────────────────────────────────────────────
Word:  _ r a _ _ _ _ e 
Guesses left:  6
Number of tries left:  6
Computer's guess:  t
────────────────────────────────────────────────────────────────────────────────────────────────────
Word:  _ r a _ t _ _ e 
Guesses left:  6
Number of tries left:  6
Computer's guess:  n
────────────────────────────────────────────────────────────────────────────────────────────────────
Word:  _ r a _ t _ _ e 
Guesses left:  5
Number of tries lef